In [1]:
import pandas as pd
import json
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
import string
import torch
import torch.optim as optim
import torch.nn as nn

In [6]:
with open('email_training_data.json') as file:
    training_data_json = json.load(file)

with open('email_testing_data.json') as file:
    testing_data_json = json.load(file)

with open('emaildata.json') as file:
    data = json.load(file)
    df = pd.DataFrame(data)

In [4]:
# # to do exploratory analysis
# including distribution of labels - visualisation
# to do correlation matrix of the features

SyntaxError: invalid syntax (525981638.py, line 2)

In [9]:
df.dtypes

Sender Address    object
Sender Name       object
Subject           object
Content           object
Category          object
dtype: object

In [5]:
emails_content = []
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
label_mapping = {'Personal': 0, 'Marketing': 1, 'Updates': 2}
labels = [label_mapping[data['Category']] for data in training_data_json]
for data in training_data_json:
  emails_content.append(data['Content'])
tokenized_emails = [tokenizer(email, padding="max_length", truncation=True, return_tensors="pt") for email in emails_content]

In [6]:
input_ids = torch.stack([email['input_ids'][0] for email in tokenized_emails])
attention_masks = torch.stack([email['attention_mask'][0] for email in tokenized_emails])
labels_tensor = torch.tensor(labels)

In [ ]:
import torch
from transformers import BertForSequenceClassification
import torch.nn.functional as F

# Load model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

# Convert labels to tensor and move to the device
labels_tensor = torch.tensor(labels).to(device)

# Define batch size
batch_size = 2

# Create DataLoader
data = torch.utils.data.TensorDataset(input_ids, attention_masks, labels_tensor)
data_loader = torch.utils.data.DataLoader(data, batch_size=batch_size)

# Define number of epochs
num_epochs = 2

# Train the model
model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, batch in enumerate(data_loader):
        input_ids_batch, attention_masks_batch, labels_batch = [t.to(device) for t in batch]

        # Forward pass
        outputs = model(input_ids_batch, attention_mask=attention_masks_batch, labels=labels_batch)
        
        # Compute loss
        loss = outputs.loss
        
        # Backward pass
        loss.backward()
        
        running_loss += loss.item()

        # Update weights every 16 batches
        if (i + 1) % 16 == 0:
            optimizer.step()
            optimizer.zero_grad()

        # Free up memory
        del input_ids_batch, attention_masks_batch, labels_batch, outputs, loss
        torch.cuda.empty_cache()

    # Print average loss for each epoch
    print(f"Epoch {epoch+1}, Loss: {running_loss / len(data_loader)}")

# Save the trained model
torch.save(model.state_dict(), 'bert_email_classification_model1.pth')


In [1]:
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Move the model to the device
# model.to(device)

# # Define optimizer and loss function
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
# criterion = torch.nn.CrossEntropyLoss()

# # Convert labels to tensor and move to the device
# labels_tensor = torch.tensor(labels).to(device)

# # Define batch size
# batch_size = 2  # Reduce batch size

# # Create DataLoader
# data = torch.utils.data.TensorDataset(input_ids, attention_masks, labels_tensor)
# data_loader = torch.utils.data.DataLoader(data, batch_size=batch_size)

# # Define number of epochs
# num_epochs = 2  # Adjust as needed

# # Train the model
# model.train()
# for epoch in range(num_epochs):
#     running_loss = 0.0
#     for batch in data_loader:
#         batch = tuple(t.to(device) for t in batch)
#         input_ids_batch, attention_masks_batch, labels_batch = batch

#         # Zero gradients
#         optimizer.zero_grad()

#         # Forward pass
#         outputs = model(input_ids_batch, attention_mask=attention_masks_batch, labels=labels_batch)
        
#         # Compute loss
#         loss = outputs.loss
        
#         # Backward pass and optimize
#         loss.backward()
#         optimizer.step()
        
#         running_loss += loss.item()

#         # Free up memory
#         del input_ids_batch, attention_masks_batch, labels_batch, outputs, loss
#         torch.cuda.empty_cache()

#     # Print average loss for each epoch
#     print(f"Epoch {epoch+1}, Loss: {running_loss / len(data_loader)}")

# # Save the trained model
# torch.save(model.state_dict(), 'bert_email_classification_model1.pth')

NameError: name 'BertForSequenceClassification' is not defined

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
model.load_state_dict(torch.load('bert_email_classification_model.pth'))
model.eval()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define batch size for testing
test_batch_size = 32  # Adjust as needed

# Define function to predict labels for a batch of input texts
def predict_labels(texts):
    # Tokenize input texts
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    input_ids = inputs['input_ids'].to(device)
    attention_masks = inputs['attention_mask'].to(device)

    # Forward pass
    with torch.no_grad():  # Disable gradient tracking during inference
        outputs = model(input_ids, attention_mask=attention_masks)
    
    # Get predicted labels
    predicted_labels = torch.argmax(outputs.logits, dim=1)
    return predicted_labels.cpu().tolist()

# Assuming you have a list of test emails stored in 'test_emails'
# Replace 'test_emails' with your actual test data
test_predicted_labels = []
test_emails_content = []
for data in testing_data_json:
  test_emails_content.append(data['content'])

# Iterate over test data in batches and predict labels
for i in range(0, len(test_emails_content), test_batch_size):
    batch_texts = test_emails_content[i:i+test_batch_size]
    batch_predicted_labels = predict_labels(batch_texts)
    test_predicted_labels.extend(batch_predicted_labels)

# Now, 'test_predicted_labels' contains the predicted labels for the test emails

In [ ]:
print(test_predicted_labels)
label_mapping = {'Personal': 0, 'Marketing': 1, 'Updates': 2}
correct_test_labels = [label_mapping[data['Category']] for data in testing_data_json]
print(correct_test_labels)

counter = 0
for i in range(len(test_predicted_labels)):
    if test_predicted_labels[i] == correct_test_labels[i]:
        counter += 1
print(f"Accuracy: {counter / len(test_predicted_labels) * 100}")